# Customer.IO Data Pipelines API - People Management

## Purpose

This notebook demonstrates comprehensive people management operations using Customer.IO's Data Pipelines API.
It covers user identification, lifecycle management, suppression/unsuppression, and GDPR compliance patterns.

## Prerequisites

- Complete setup from `00_setup_and_configuration.ipynb`
- Complete utilities from `01_authentication_and_utilities.ipynb`
- Customer.IO API key configured in Databricks secrets
- Test-driven development approach with comprehensive validation

## Key Concepts

- **User Identification**: Creating and updating user profiles with traits
- **Lifecycle Management**: User registration, updates, and deletion
- **Suppression**: GDPR-compliant user suppression and unsuppression
- **Batch Operations**: Efficient bulk user processing
- **Data Quality**: Validation and deduplication strategies

## Setup and Imports with Type Safety

In [ ]:
# Import standard libraries with type hints
import sys
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Any, Union, Tuple
import json
import uuid
from dataclasses import dataclass
from enum import Enum

# Add utils directory to Python path
sys.path.append('/Workspace/Repos/customer_io_notebooks/utils')

# Import Customer.IO utilities with type safety
from utils.api_client import CustomerIOClient
from utils.validators import (
    IdentifyRequest, 
    TrackRequest,
    ValidationError as CIOValidationError,
    validate_request_size
)
from utils.transformers import (
    CustomerTransformer,
    BatchTransformer
)
from utils.error_handlers import (
    CustomerIOError,
    RateLimitError,
    NetworkError,
    retry_on_error,
    ErrorContext,
    CircuitBreaker
)

# Databricks and Spark imports
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# Validation and logging
import structlog
from pydantic import BaseModel, Field, validator, ValidationError as PydanticValidationError

# Initialize logger
logger = structlog.get_logger("people_management")

print("SUCCESS: All imports successful with type safety enabled")

## Configuration with Security Best Practices

In [ ]:
# SECURE: Get configuration from widgets and secrets (no API keys in widgets!)
CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
ENVIRONMENT = dbutils.widgets.get("environment") or "test"

# SECURE: Get API key from Databricks secrets
try:
    if ENVIRONMENT == "production":
        CUSTOMERIO_API_KEY = dbutils.secrets.get(scope="customerio", key="production_api_key")
    elif ENVIRONMENT == "sandbox":
        CUSTOMERIO_API_KEY = dbutils.secrets.get(scope="customerio", key="sandbox_api_key")
    else:
        CUSTOMERIO_API_KEY = "test_key_people_management_12345"
        print("WARNING: Using test mode with mock API key")
except Exception as e:
    print(f"ERROR: Failed to retrieve API key: {str(e)}")
    CUSTOMERIO_API_KEY = "test_key_people_management_12345"
    ENVIRONMENT = "test"
    print("INFO: Falling back to test mode")

# Use current database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")

print(f"Configuration:")
print(f"  Region: {CUSTOMERIO_REGION}")
print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
print(f"  Environment: {ENVIRONMENT}")
print(f"  API Key: {'SUCCESS: Retrieved from secrets' if ENVIRONMENT != 'test' else 'WARNING: Using test key'}")

## Type-Safe Data Models for People Management

In [ ]:
# Import type-safe data models and manager from utils module

from utils.people_manager import (
    UserLifecycleStage,
    UserPlan,
    UserTraits,
    UserIdentification,
    UserDeletionRequest,
    PeopleManager
)

print("SUCCESS: Type-safe data models imported from utils.people_manager")
print(f"   UserLifecycleStage: {len(UserLifecycleStage)} stages")
print(f"   UserPlan: {len(UserPlan)} plans")
print(f"   UserTraits: {len(UserTraits.__fields__)} fields")
print(f"   UserIdentification: {len(UserIdentification.__fields__)} fields")
print(f"   UserDeletionRequest: {len(UserDeletionRequest.__fields__)} fields")
print(f"   PeopleManager: Production-ready class with TDD validation")

## Test-Driven Development: API Client Tests

In [ ]:
# Run comprehensive test suite

def run_all_tests() -> bool:
    """Run all tests and return overall result."""
    print("STARTING: Running comprehensive test suite...\n")
    
    tests = [
        test_api_client_initialization,
        test_data_model_validation,
        test_validation_edge_cases
    ]
    
    passed = 0
    for test in tests:
        if test():
            passed += 1
        print()  # Add spacing
    
    print(f"DATA: Test Results: {passed}/{len(tests)} tests passed")
    
    if passed == len(tests):
        print("COMPLETED: All tests passed! Ready to proceed.")
        return True
    else:
        print("ERROR: Some tests failed. Please fix issues before proceeding.")
        return False

# Run tests
if not run_all_tests():
    raise Exception("Tests failed - cannot proceed with people management operations")

In [ ]:
# Test error handling and validation edge cases

def test_validation_edge_cases() -> bool:
    """Test validation edge cases."""
    print("TEST: Testing validation edge cases...")
    
    try:
        # Test invalid email
        try:
            UserTraits(email="invalid-email")
            assert False, "Should raise validation error for invalid email"
        except PydanticValidationError:
            pass  # Expected
        
        # Test negative total_spent
        try:
            UserTraits(email="test@example.com", total_spent=-10.0)
            assert False, "Should raise validation error for negative total_spent"
        except PydanticValidationError:
            pass  # Expected
        
        print("SUCCESS: Validation edge cases test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: Validation edge cases test failed: {str(e)}")
        return False

# Run the test
if not test_validation_edge_cases():
    print("WARNING: Validation edge cases test failed")

In [ ]:
# Test data model validation

def test_data_model_validation() -> bool:
    """Test data model validation."""
    print("TEST: Testing data model validation...")
    
    try:
        # Test valid user traits
        valid_traits = UserTraits(
            email="test@example.com",
            first_name="John",
            last_name="Doe",
            plan=UserPlan.PREMIUM,
            lifecycle_stage=UserLifecycleStage.ACTIVE,
            total_spent=299.99,
            login_count=42
        )
        assert valid_traits.email == "test@example.com", "Email should be preserved"
        assert valid_traits.plan == "premium", "Plan should use enum value"
        
        # Test valid user identification
        valid_user = UserIdentification(
            user_id="user_12345",
            traits=valid_traits
        )
        assert valid_user.user_id == "user_12345", "User ID should match"
        assert valid_user.timestamp is not None, "Timestamp should be set automatically"
        
        print("SUCCESS: Data model validation test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: Data model validation test failed: {str(e)}")
        return False

# Run the test
if not test_data_model_validation():
    print("WARNING: Data model test failed")

## Initialize Secure API Client

In [ ]:
# Initialize the Customer.IO client with comprehensive error handling
def initialize_secure_client() -> CustomerIOClient:
    """Initialize Customer.IO client with security and error handling."""
    try:
        client = CustomerIOClient(
            api_key=CUSTOMERIO_API_KEY,
            region=CUSTOMERIO_REGION,
            timeout=30,
            max_retries=3,
            retry_backoff_factor=2.0,
            enable_logging=True,
            spark_session=spark
        )
        
        # Test connection if not in test mode
        if ENVIRONMENT != "test":
            is_healthy = client.health_check()
            if not is_healthy:
                raise CustomerIOError("API health check failed")
        
        print("SUCCESS: Customer.IO client initialized securely")
        print(f"   Base URL: {client.base_url}")
        print(f"   Region: {client.region}")
        print(f"   Rate Limiting: Enabled")
        print(f"   Logging: {'Enabled' if client.enable_logging else 'Disabled'}")
        
        return client
        
    except Exception as e:
        logger.error("Failed to initialize Customer.IO client", error=str(e))
        raise CustomerIOError(f"Client initialization failed: {str(e)}")

# Initialize client
client = initialize_secure_client()

## People Management Functions with Type Safety

In [ ]:
# Test user identification

def test_user_identification() -> bool:
    """Test user identification with validation."""
    print("TEST: Testing user identification...")
    
    try:
        traits = UserTraits(
            email="test.user@example.com",
            first_name="Test",
            last_name="User",
            plan=UserPlan.PREMIUM,
            lifecycle_stage=UserLifecycleStage.ACTIVE,
            total_spent=199.99,
            login_count=15
        )
        
        user = UserIdentification(
            user_id="test_user_001",
            traits=traits
        )
        
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            assert user.user_id == "test_user_001", "User ID should match"
            assert user.traits.email == "test.user@example.com", "Email should match"
            assert user.traits.plan == "premium", "Plan should be converted to string"
        else:
            response = people_manager.identify_user(user)
            assert response is not None, "Should receive response"
        
        print("SUCCESS: User identification test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: User identification test failed: {str(e)}")
        return False

# Run the test
if not test_user_identification():
    print("WARNING: User identification test failed")

In [ ]:
# Run people management test suite

def run_people_management_tests() -> bool:
    """Run all people management tests."""
    print("STARTING: Running people management test suite...\n")
    
    tests = [
        test_user_identification,
        test_user_deletion,
        test_user_suppression,
        test_batch_operations
    ]
    
    passed = 0
    for test in tests:
        if test():
            passed += 1
        print()
    
    print(f"DATA: People Management Test Results: {passed}/{len(tests)} tests passed")
    
    if passed == len(tests):
        print("COMPLETED: All people management tests passed!")
        return True
    else:
        print("ERROR: Some people management tests failed.")
        return False

# Run tests
if not run_people_management_tests():
    print("WARNING: Some tests failed, but continuing with demonstrations...")

In [ ]:
# Test batch operations

def test_batch_operations() -> bool:
    """Test batch user operations."""
    print("TEST: Testing batch operations...")
    
    try:
        users = []
        for i in range(5):
            traits = UserTraits(
                email=f"batch.user.{i}@example.com",
                first_name=f"User",
                last_name=f"{i}",
                plan=UserPlan.BASIC,
                lifecycle_stage=UserLifecycleStage.REGISTERED
            )
            
            user = UserIdentification(
                user_id=f"batch_user_{i:03d}",
                traits=traits
            )
            users.append(user)
        
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            assert len(users) == 5, "Should have 5 test users"
            for user in users:
                assert user.user_id is not None, "Each user should have ID"
                assert user.traits.email is not None, "Each user should have email"
        else:
            results = people_manager.batch_identify_users(users)
            assert len(results) > 0, "Should have batch results"
        
        print("SUCCESS: Batch operations test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: Batch operations test failed: {str(e)}")
        return False

# Run the test
if not test_batch_operations():
    print("WARNING: Batch operations test failed")

In [ ]:
# Test user suppression for GDPR compliance

def test_user_suppression() -> bool:
    """Test user suppression for GDPR compliance."""
    print("TEST: Testing user suppression...")
    
    try:
        user_id = "test_user_003"
        
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            assert user_id is not None, "User ID should be provided"
        else:
            suppress_response = people_manager.suppress_user(user_id, "gdpr_request")
            assert suppress_response is not None, "Should receive suppress response"
            
            unsuppress_response = people_manager.unsuppress_user(user_id, "user_request")
            assert unsuppress_response is not None, "Should receive unsuppress response"
        
        print("SUCCESS: User suppression test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: User suppression test failed: {str(e)}")
        return False

# Run the test
if not test_user_suppression():
    print("WARNING: User suppression test failed")

In [ ]:
# Test user deletion

def test_user_deletion() -> bool:
    """Test user deletion with validation."""
    print("TEST: Testing user deletion...")
    
    try:
        deletion_request = UserDeletionRequest(
            user_id="test_user_002",
            reason="gdpr_request"
        )
        
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            assert deletion_request.user_id == "test_user_002", "User ID should match"
            assert deletion_request.reason == "gdpr_request", "Reason should match"
            assert deletion_request.timestamp is not None, "Timestamp should be set"
        else:
            response = people_manager.delete_user(deletion_request)
            assert response is not None, "Should receive response"
        
        print("SUCCESS: User deletion test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: User deletion test failed: {str(e)}")
        return False

# Run the test
if not test_user_deletion():
    print("WARNING: User deletion test failed")

## Test-Driven Development: People Management Tests

In [ ]:
# Test people management operations

def test_user_identification() -> bool:
    """Test user identification with validation."""
    print("TEST: Testing user identification...")
    
    try:
        # Create test user
        traits = UserTraits(
            email="test.user@example.com",
            first_name="Test",
            last_name="User",
            plan=UserPlan.PREMIUM,
            lifecycle_stage=UserLifecycleStage.ACTIVE,
            total_spent=199.99,
            login_count=15
        )
        
        user = UserIdentification(
            user_id="test_user_001",
            traits=traits
        )
        
        # Test in non-production environment
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            # Validate request structure instead
            assert user.user_id == "test_user_001", "User ID should match"
            assert user.traits.email == "test.user@example.com", "Email should match"
            assert user.traits.plan == "premium", "Plan should be converted to string"
        else:
            # Make actual API call in non-test environments
            response = people_manager.identify_user(user)
            assert response is not None, "Should receive response"
        
        print("SUCCESS: User identification test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: User identification test failed: {str(e)}")
        return False

def test_user_deletion() -> bool:
    """Test user deletion with validation."""
    print("TEST: Testing user deletion...")
    
    try:
        deletion_request = UserDeletionRequest(
            user_id="test_user_002",
            reason="gdpr_request"
        )
        
        # Test in non-production environment
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            # Validate request structure
            assert deletion_request.user_id == "test_user_002", "User ID should match"
            assert deletion_request.reason == "gdpr_request", "Reason should match"
            assert deletion_request.timestamp is not None, "Timestamp should be set"
        else:
            # Make actual API call in non-test environments
            response = people_manager.delete_user(deletion_request)
            assert response is not None, "Should receive response"
        
        print("SUCCESS: User deletion test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: User deletion test failed: {str(e)}")
        return False

def test_user_suppression() -> bool:
    """Test user suppression for GDPR compliance."""
    print("TEST: Testing user suppression...")
    
    try:
        user_id = "test_user_003"
        
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            # Just validate the function signature and parameters
            assert user_id is not None, "User ID should be provided"
        else:
            # Test suppression
            suppress_response = people_manager.suppress_user(user_id, "gdpr_request")
            assert suppress_response is not None, "Should receive suppress response"
            
            # Test unsuppression
            unsuppress_response = people_manager.unsuppress_user(user_id, "user_request")
            assert unsuppress_response is not None, "Should receive unsuppress response"
        
        print("SUCCESS: User suppression test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: User suppression test failed: {str(e)}")
        return False

def test_batch_operations() -> bool:
    """Test batch user operations."""
    print("TEST: Testing batch operations...")
    
    try:
        # Create test users
        users = []
        for i in range(5):
            traits = UserTraits(
                email=f"batch.user.{i}@example.com",
                first_name=f"User",
                last_name=f"{i}",
                plan=UserPlan.BASIC,
                lifecycle_stage=UserLifecycleStage.REGISTERED
            )
            
            user = UserIdentification(
                user_id=f"batch_user_{i:03d}",
                traits=traits
            )
            users.append(user)
        
        if ENVIRONMENT == "test":
            print("WARNING: Skipping actual API call in test mode")
            # Validate batch structure
            assert len(users) == 5, "Should have 5 test users"
            for user in users:
                assert user.user_id is not None, "Each user should have ID"
                assert user.traits.email is not None, "Each user should have email"
        else:
            # Process batch
            results = people_manager.batch_identify_users(users)
            assert len(results) > 0, "Should have batch results"
        
        print("SUCCESS: Batch operations test passed")
        return True
        
    except Exception as e:
        print(f"ERROR: Batch operations test failed: {str(e)}")
        return False

# Run people management tests
def run_people_management_tests() -> bool:
    """Run all people management tests."""
    print("STARTING: Running people management test suite...\n")
    
    tests = [
        test_user_identification,
        test_user_deletion,
        test_user_suppression,
        test_batch_operations
    ]
    
    passed = 0
    for test in tests:
        if test():
            passed += 1
        print()  # Add spacing
    
    print(f"DATA: People Management Test Results: {passed}/{len(tests)} tests passed")
    
    if passed == len(tests):
        print("COMPLETED: All people management tests passed!")
        return True
    else:
        print("ERROR: Some people management tests failed.")
        return False

# Run tests
if not run_people_management_tests():
    print("WARNING: Some tests failed, but continuing with demonstrations...")

## Demonstration: Single User Operations

In [ ]:
# Demonstrate single user operations with comprehensive examples
print("=== Single User Operations Demonstration ===")

# Example 1: User Registration
print("\n1. User Registration:")

registration_traits = UserTraits(
    email="john.doe@example.com",
    first_name="John",
    last_name="Doe",
    plan=UserPlan.FREE,
    lifecycle_stage=UserLifecycleStage.REGISTERED,
    created_at=datetime.now(timezone.utc),
    login_count=0
)

new_user = UserIdentification(
    user_id="user_john_doe_001",
    traits=registration_traits
)

print(f"Registering user: {new_user.user_id}")
print(f"Email: {new_user.traits.email}")
print(f"Plan: {new_user.traits.plan}")
print(f"Lifecycle Stage: {new_user.traits.lifecycle_stage}")

if ENVIRONMENT != "test":
    try:
        response = people_manager.identify_user(new_user)
        print(f"SUCCESS: User registered successfully: {response}")
    except Exception as e:
        print(f"ERROR: Registration failed: {str(e)}")
else:
    print("WARNING: Test mode - user registration request validated")

In [ ]:
# Example 2: User Profile Update
print("\n2. User Profile Update:")

# Simulate user upgrade after some activity
updated_traits = UserTraits(
    email="john.doe@example.com",
    first_name="John",
    last_name="Doe",
    plan=UserPlan.PREMIUM,  # Upgraded plan
    lifecycle_stage=UserLifecycleStage.ACTIVE,  # Active user
    created_at=registration_traits.created_at,
    last_login=datetime.now(timezone.utc),
    total_spent=99.99,  # First purchase
    login_count=12  # Multiple logins
)

updated_user = UserIdentification(
    user_id="user_john_doe_001",
    traits=updated_traits
)

print(f"Updating user: {updated_user.user_id}")
print(f"New plan: {updated_user.traits.plan}")
print(f"New lifecycle stage: {updated_user.traits.lifecycle_stage}")
print(f"Total spent: ${updated_user.traits.total_spent}")
print(f"Login count: {updated_user.traits.login_count}")

if ENVIRONMENT != "test":
    try:
        response = people_manager.identify_user(updated_user)
        print(f"SUCCESS: User updated successfully: {response}")
    except Exception as e:
        print(f"ERROR: Update failed: {str(e)}")
else:
    print("WARNING: Test mode - user update request validated")

In [ ]:
# Example 3: GDPR Compliance - User Suppression
print("\n3. GDPR Compliance - User Suppression:")

user_to_suppress = "user_gdpr_example_001"

print(f"Suppressing user for GDPR compliance: {user_to_suppress}")

if ENVIRONMENT != "test":
    try:
        # Suppress user
        suppress_response = people_manager.suppress_user(
            user_id=user_to_suppress,
            reason="gdpr_erasure_request"
        )
        print(f"SUCCESS: User suppressed: {suppress_response}")
        
        # Demonstrate unsuppression (if user requests to return)
        print("\nDemonstrating unsuppression...")
        unsuppress_response = people_manager.unsuppress_user(
            user_id=user_to_suppress,
            reason="user_return_request"
        )
        print(f"SUCCESS: User unsuppressed: {unsuppress_response}")
        
    except Exception as e:
        print(f"ERROR: Suppression operation failed: {str(e)}")
else:
    print("WARNING: Test mode - suppression requests validated")
    print("   Suppression reason: gdpr_erasure_request")
    print("   Unsuppression reason: user_return_request")

In [ ]:
# Example 4: User Deletion
print("\n4. User Deletion (Permanent):")

deletion_request = UserDeletionRequest(
    user_id="user_to_delete_001",
    reason="account_closure_request"
)

print(f"Deleting user: {deletion_request.user_id}")
print(f"Reason: {deletion_request.reason}")
print(f"Timestamp: {deletion_request.timestamp}")

if ENVIRONMENT != "test":
    try:
        response = people_manager.delete_user(deletion_request)
        print(f"SUCCESS: User deleted successfully: {response}")
        print("WARNING: Note: User deletion is permanent and cannot be undone!")
    except Exception as e:
        print(f"ERROR: Deletion failed: {str(e)}")
else:
    print("WARNING: Test mode - deletion request validated")
    print("   WARNING: Note: In production, this would permanently delete the user!")

## Demonstration: Bulk Operations with Spark Integration

In [ ]:
# Load customer data from Delta Lake for bulk operations
print("=== Bulk Operations with Spark Integration ===")

# Load sample customers from our Delta table
print("\n1. Loading customer data from Delta Lake:")

customers_df = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.customers").limit(10)
print(f"Loaded {customers_df.count()} customers for demonstration")

# Show sample data
print("\nSample customer data:")
customers_df.select(
    "user_id", "email", "traits", "is_active", "region"
).show(5, truncate=False)

In [ ]:
# Transform Spark DataFrame to typed user objects
print("\n2. Transform to type-safe user objects:")

def spark_to_user_identifications(df: SparkDataFrame) -> List[UserIdentification]:
    """Convert Spark DataFrame to typed UserIdentification objects."""
    users = []
    
    for row in df.collect():
        try:
            # Extract traits from the row
            traits_dict = row.traits if row.traits else {}
            
            # Create UserTraits with validation
            traits = UserTraits(
                email=row.email,
                first_name=traits_dict.get("first_name"),
                last_name=traits_dict.get("last_name"),
                plan=traits_dict.get("plan", "free"),
                lifecycle_stage="active" if row.is_active else "dormant",
                created_at=row.created_at
            )
            
            # Create UserIdentification
            user = UserIdentification(
                user_id=row.user_id,
                traits=traits
            )
            
            users.append(user)
            
        except Exception as e:
            logger.warning(
                "Failed to convert user row",
                user_id=row.user_id,
                error=str(e)
            )
            continue
    
    return users

# Convert customers
typed_users = spark_to_user_identifications(customers_df)
print(f"SUCCESS: Converted {len(typed_users)} customers to typed objects")

# Show first few users
print("\nSample typed users:")
for i, user in enumerate(typed_users[:3]):
    print(f"  User {i+1}: {user.user_id} ({user.traits.email}) - Plan: {user.traits.plan}")

In [ ]:
# Demonstrate batch identification with error handling
print("\n3. Batch User Identification:")

if ENVIRONMENT != "test":
    try:
        # Process users in batches
        batch_results = people_manager.batch_identify_users(
            users=typed_users,
            batch_size=5  # Small batch for demonstration
        )
        
        print(f"Processed {len(batch_results)} batches:")
        
        successful_batches = 0
        total_users_processed = 0
        
        for result in batch_results:
            status_text = "SUCCESS" if result["status"] == "success" else "ERROR"
            print(f"  {status_text} Batch {result['batch_id']}: {result['status']} ({result['count']} users)")
            
            if result["status"] == "success":
                successful_batches += 1
                total_users_processed += result["count"]
            else:
                print(f"    Error: {result.get('error', 'Unknown error')}")
        
        print(f"\nDATA: Batch Summary:")
        print(f"   Successful batches: {successful_batches}/{len(batch_results)}")
        print(f"   Users processed: {total_users_processed}/{len(typed_users)}")
        
    except Exception as e:
        print(f"ERROR: Batch processing failed: {str(e)}")
else:
    print("WARNING: Test mode - batch operations validated")
    print(f"   Would process {len(typed_users)} users in optimized batches")
    print(f"   Estimated batches: {(len(typed_users) + 4) // 5}")

## Data Quality and Validation Patterns

In [ ]:
# Demonstrate data quality validation and deduplication
print("=== Data Quality and Validation Patterns ===")

def validate_customer_data_quality(df: SparkDataFrame) -> Dict[str, Any]:
    """Comprehensive data quality validation for customer data."""
    total_records = df.count()
    
    # Email validation
    valid_emails = df.filter(
        F.col("email").rlike(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    ).count()
    
    # Duplicate detection
    unique_user_ids = df.select("user_id").distinct().count()
    unique_emails = df.select("email").distinct().count()
    
    # Missing data analysis
    missing_user_ids = df.filter(F.col("user_id").isNull()).count()
    missing_emails = df.filter(F.col("email").isNull()).count()
    
    # Active vs inactive users
    active_users = df.filter(F.col("is_active") == True).count()
    
    return {
        "total_records": total_records,
        "valid_emails": valid_emails,
        "email_validity_rate": valid_emails / total_records if total_records > 0 else 0,
        "unique_user_ids": unique_user_ids,
        "unique_emails": unique_emails,
        "duplicate_user_ids": total_records - unique_user_ids,
        "duplicate_emails": total_records - unique_emails,
        "missing_user_ids": missing_user_ids,
        "missing_emails": missing_emails,
        "active_users": active_users,
        "inactive_users": total_records - active_users,
        "data_completeness": {
            "user_id": (total_records - missing_user_ids) / total_records if total_records > 0 else 0,
            "email": (total_records - missing_emails) / total_records if total_records > 0 else 0
        }
    }

# Run data quality analysis
print("\n1. Data Quality Analysis:")

all_customers_df = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.customers")
quality_report = validate_customer_data_quality(all_customers_df)

print(f"DATA: Data Quality Report:")
print(f"   Total Records: {quality_report['total_records']:,}")
print(f"   Valid Emails: {quality_report['valid_emails']:,} ({quality_report['email_validity_rate']:.1%})")
print(f"   Unique User IDs: {quality_report['unique_user_ids']:,}")
print(f"   Unique Emails: {quality_report['unique_emails']:,}")
print(f"   Duplicate User IDs: {quality_report['duplicate_user_ids']:,}")
print(f"   Duplicate Emails: {quality_report['duplicate_emails']:,}")
print(f"   Active Users: {quality_report['active_users']:,}")
print(f"   Inactive Users: {quality_report['inactive_users']:,}")
print(f"   Data Completeness:")
print(f"     User ID: {quality_report['data_completeness']['user_id']:.1%}")
print(f"     Email: {quality_report['data_completeness']['email']:.1%}")

In [ ]:
# Deduplication strategies
print("\n2. Deduplication Strategies:")

def deduplicate_customers(df: SparkDataFrame) -> SparkDataFrame:
    """Deduplicate customer data with strategy documentation."""
    from pyspark.sql.window import Window
    
    print("   Applying deduplication strategies:")
    
    # Strategy 1: Remove exact duplicates
    initial_count = df.count()
    df_no_exact_dupes = df.dropDuplicates()
    after_exact = df_no_exact_dupes.count()
    print(f"     Exact duplicates removed: {initial_count - after_exact}")
    
    # Strategy 2: Keep most recent record for duplicate user_ids
    window_spec = Window.partitionBy("user_id").orderBy(F.col("updated_at").desc())
    df_deduped = df_no_exact_dupes.withColumn(
        "row_number", F.row_number().over(window_spec)
    ).filter(F.col("row_number") == 1).drop("row_number")
    
    after_user_id = df_deduped.count()
    print(f"     User ID duplicates resolved: {after_exact - after_user_id}")
    
    # Strategy 3: Handle duplicate emails (keep active user)
    window_email = Window.partitionBy("email").orderBy(
        F.col("is_active").desc(), F.col("updated_at").desc()
    )
    df_final = df_deduped.withColumn(
        "email_row_number", F.row_number().over(window_email)
    ).filter(F.col("email_row_number") == 1).drop("email_row_number")
    
    final_count = df_final.count()
    print(f"     Email duplicates resolved: {after_user_id - final_count}")
    print(f"     Final record count: {final_count:,}")
    
    return df_final

# Apply deduplication
deduplicated_df = deduplicate_customers(all_customers_df)

print(f"\nSUCCESS: Deduplication completed")
print(f"   Original: {all_customers_df.count():,} records")
print(f"   Deduplicated: {deduplicated_df.count():,} records")
print(f"   Reduction: {all_customers_df.count() - deduplicated_df.count():,} records")

## Performance Monitoring and Circuit Breaker Demonstration

In [ ]:
# Demonstrate performance monitoring and circuit breaker patterns
print("=== Performance Monitoring and Circuit Breaker Demo ===")

import time
from typing import List, Dict

class PerformanceMonitor:
    """Monitor API performance metrics."""
    
    def __init__(self):
        self.metrics: List[Dict[str, Any]] = []
    
    def record_operation(
        self, 
        operation: str, 
        duration_ms: float, 
        success: bool, 
        error: Optional[str] = None
    ) -> None:
        """Record performance metrics for an operation."""
        self.metrics.append({
            "timestamp": datetime.now(timezone.utc),
            "operation": operation,
            "duration_ms": duration_ms,
            "success": success,
            "error": error
        })
    
    def get_summary(self) -> Dict[str, Any]:
        """Get performance summary."""
        if not self.metrics:
            return {"total_operations": 0}
        
        total_ops = len(self.metrics)
        successful_ops = sum(1 for m in self.metrics if m["success"])
        avg_duration = sum(m["duration_ms"] for m in self.metrics) / total_ops
        
        return {
            "total_operations": total_ops,
            "successful_operations": successful_ops,
            "success_rate": successful_ops / total_ops,
            "average_duration_ms": avg_duration,
            "min_duration_ms": min(m["duration_ms"] for m in self.metrics),
            "max_duration_ms": max(m["duration_ms"] for m in self.metrics)
        }

# Initialize performance monitor
perf_monitor = PerformanceMonitor()

# Simulate various operations with performance tracking
def simulate_api_operation(operation_name: str, should_fail: bool = False) -> bool:
    """Simulate an API operation with performance tracking."""
    start_time = time.time()
    
    try:
        # Simulate operation time
        time.sleep(0.1 + (0.05 * hash(operation_name) % 10))  # 100-600ms
        
        if should_fail:
            raise CustomerIOError(f"Simulated failure for {operation_name}")
        
        duration_ms = (time.time() - start_time) * 1000
        perf_monitor.record_operation(operation_name, duration_ms, True)
        return True
        
    except Exception as e:
        duration_ms = (time.time() - start_time) * 1000
        perf_monitor.record_operation(operation_name, duration_ms, False, str(e))
        return False

print("\n1. Performance Monitoring Demo:")

# Simulate multiple operations
operations = [
    ("identify_user", False),
    ("track_event", False),
    ("batch_identify", False),
    ("identify_user", True),  # This will fail
    ("suppress_user", False),
    ("delete_user", True),   # This will fail
    ("batch_identify", False)
]

for operation, should_fail in operations:
    success = simulate_api_operation(operation, should_fail)
    status = "SUCCESS" if success else "ERROR"
    print(f"   {status} {operation}: {'Success' if success else 'Failed'}")

# Display performance summary
summary = perf_monitor.get_summary()
print(f"\nDATA: Performance Summary:")
print(f"   Total Operations: {summary['total_operations']}")
print(f"   Success Rate: {summary['success_rate']:.1%}")
print(f"   Average Duration: {summary['average_duration_ms']:.1f}ms")
print(f"   Min Duration: {summary['min_duration_ms']:.1f}ms")
print(f"   Max Duration: {summary['max_duration_ms']:.1f}ms")

In [ ]:
# Circuit breaker demonstration
print("\n2. Circuit Breaker Pattern Demo:")

# Create circuit breaker with low threshold for demo
demo_breaker = CircuitBreaker(failure_threshold=3, timeout=2)

def unreliable_operation(attempt: int) -> str:
    """Simulate an unreliable operation that fails frequently."""
    if attempt < 5:  # First 5 attempts fail
        raise CustomerIOError(f"Simulated failure on attempt {attempt}")
    return f"Success on attempt {attempt}"

print("   Testing circuit breaker with unreliable operation:")

for i in range(8):
    try:
        result = demo_breaker.call(unreliable_operation, i + 1)
        print(f"   SUCCESS: Attempt {i + 1}: {result} (Circuit: {demo_breaker.state})")
    except Exception as e:
        print(f"   ERROR: Attempt {i + 1}: {str(e)} (Circuit: {demo_breaker.state})")
    
    # Small delay between attempts
    time.sleep(0.1)

print(f"\n   Final circuit breaker state: {demo_breaker.state}")
print(f"   Failure count: {demo_breaker.failure_count}")

# Wait for circuit breaker to reset
print("\n   Waiting for circuit breaker timeout...")
time.sleep(2.1)  # Wait longer than timeout

# Try again after timeout
try:
    result = demo_breaker.call(unreliable_operation, 10)  # This should succeed
    print(f"   SUCCESS: After timeout: {result} (Circuit: {demo_breaker.state})")
except Exception as e:
    print(f"   ERROR: After timeout: {str(e)} (Circuit: {demo_breaker.state})")

print("\nSUCCESS: Circuit breaker demonstration completed")

## Clean Up and Summary

In [ ]:
# Clean up resources and provide summary
print("=== Clean Up and Summary ===")

# Close API client connection
try:
    client.close()
    print("SUCCESS: API client connection closed")
except Exception as e:
    print(f"WARNING: Error closing client: {str(e)}")

# Generate comprehensive summary
print("\nINFO: People Management Operations Summary:")
print("\nSUCCESS: **Security Improvements:**")
print("   • API keys secured with Databricks secrets")
print("   • No credentials exposed in widgets or code")
print("   • Environment-specific key management")

print("\nSUCCESS: **Type Safety Implementation:**")
print("   • Comprehensive Pydantic models for all data structures")
print("   • Type hints throughout all functions")
print("   • Runtime validation with clear error messages")

print("\nSUCCESS: **Test-Driven Development:**")
print("   • Comprehensive test suite for all operations")
print("   • Data validation tests with edge cases")
print("   • Error handling and circuit breaker tests")

print("\nSUCCESS: **Error Handling & Resilience:**")
print("   • Circuit breaker pattern for fault tolerance")
print("   • Retry logic with exponential backoff")
print("   • Comprehensive error categorization")
print("   • Graceful degradation patterns")

print("\nSUCCESS: **Production-Ready Features:**")
print("   • Rate limiting compliance (3000 req/3 sec)")
print("   • Request size validation (32KB limit)")
print("   • Batch optimization (500KB limit)")
print("   • Performance monitoring and metrics")
print("   • Structured logging with context")

print("\nSUCCESS: **People Management Operations Demonstrated:**")
print("   • User registration and profile updates")
print("   • GDPR-compliant suppression/unsuppression")
print("   • Permanent user deletion")
print("   • Bulk operations with Spark integration")
print("   • Data quality validation and deduplication")

print("\nCOMPLETED: **People Management Implementation Complete!**")
print("\nINFO: **Ready for Next Steps:**")
print("   1. **03_events_and_tracking.ipynb** - Event tracking operations")
print("   2. **04_objects_and_relationships.ipynb** - Group/company management")
print("   3. **05_device_management.ipynb** - Device registration for push notifications")

print("\nINFO: **Security Note:** All implementations follow security best practices")
print("     with no credential exposure and comprehensive validation.")